In [4]:
#1.2?
class WeightedGraph:

    def __init__(self,nodes):
        self.graph=[]
        self.weights={}
        for node in range(nodes):
            self.graph.append([])

    def add_node(self,node):
        self.graph[node]=[]

    def add_edge(self, node1, node2, weight):
        if node2 not in self.graph[node1]:
            self.graph[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def get_weights(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def are_connected(self, node1, node2):
        for neighbour in self.graph[node1]:
            if neighbour == node2:
                return True
        return False

    def get_neighbors(self, node):
        return self.graph[node]

    def get_number_of_nodes(self,):
        return len(self.graph)
    
    def get_nodes(self,):
        return [i for i in range(len(self.graph))]

In [111]:
def BellmanFord(g,src,k):
    dist = {}
    pred = {}
    dist2 = {}
    pred2 = {}
    
    #initializing distance of all nodes to infinity and dist of 1st node as 0
    for i in range(g.get_number_of_nodes()):
        dist[i] = float("inf")
        pred[i] = [src]
        dist2[i] = float("inf")
        pred2[i] = [src]
        
    dist[src] = 0
    dist2[src] = 0
    
    
    for _ in range(k):
        for node in g.get_nodes():
            for n in g.get_neighbors(node):
                weight = g.get_weights(node,n)
                if weight != None:
                    if dist[node] + weight < dist[n]:
                        dist[n] = dist[node] + weight
                        if node not in pred[n]:
                            pred[n].append(node)
                            pred[n].append(n)
                    if dist2[node] + weight < dist2[n]:
                        dist2[n] = dist2[node] + weight
                        if node not in pred2[n]:
                            pred2[n].append(node)
                            pred2[n].append(n)
    
    #negative cycle
    
    for node in g.get_nodes():
        for n in g.get_neighbors(node):
            weight = g.get_weights(node,n)
            if weight != None:
                if dist2[node] + weight < dist2[n]:
                    dist2[n] = dist2[node] + weight
                    if node not in pred2[n]:
                        pred2[n].append(node)
                    #if n not in pred[n]:
                        pred2[n].append(n)

    '''for node in g.get_nodes():
            for n in g.get_neighbors(node):
                weight = g.get_weights(node,n)
                if weight != None:
                    if dist[node] + weight > dist[n]:
                        raise ValueError("Negative cycle")'''
    
    if dist == dist2:
        return dist, pred
    else:
        raise ValueError("Negative cycle")

In [109]:
# Test Case 1: Basic Test Case
graph1 = WeightedGraph(3)
graph1.add_edge(0, 1, 2)
graph1.add_edge(0, 2, 4)
graph1.add_edge(1, 2, 2)

source1 = 0
k1 = 2

distance1, path1 = BellmanFord(graph1, source1, k1)
print("Test Case 1 - Distance:", distance1)
print("Test Case 1 - Path:", path1)

# Test Case 2: Graph with Negative Edge Weight
graph2 = WeightedGraph(4)
graph2.add_edge(0, 1, 1)
graph2.add_edge(1, 2, -1)
graph2.add_edge(2, 3, 2)

source2 = 0
k2 = 3

distance2, path2 = BellmanFord(graph2, source2, k2)
print("Test Case 2 - Distance:", distance2)
print("Test Case 2 - Path:", path2)

# Test Case 3: Graph with Negative Cycle
graph3 = WeightedGraph(4)
graph3.add_edge(0, 1, 1)
graph3.add_edge(1, 2, -3)
graph3.add_edge(2, 0, 2)
graph3.add_edge(2, 3, 1)

source3 = 0
k3 = 3

try:
    distance3, path3 = BellmanFord(graph3, source3, k3)
    print("Test Case 3 - Distance:", distance3)
    print("Test Case 3 - Path:", path3)
except ValueError as e:
    print("Test Case 3 - Error:", e)

# Test Case 4: Graph with Disconnected Components
graph4 = WeightedGraph(5)
graph4.add_edge(0, 1, 1)
graph4.add_edge(1, 2, 2)
graph4.add_edge(3, 4, 3)

source4 = 0
k4 = 3

distance4, path4 = BellmanFord(graph4, source4, k4)
print("Test Case 4 - Distance:", distance4)
print("Test Case 4 - Path:", path4)


{0: 0, 1: 2, 2: 4}
{0: 0, 1: 2, 2: 4}
Test Case 1 - Distance: {0: 0, 1: 2, 2: 4}
Test Case 1 - Path: {0: [0], 1: [0], 2: [0]}
{0: 0, 1: 1, 2: 0, 3: 2}
{0: 0, 1: 1, 2: 0, 3: 2}
Test Case 2 - Distance: {0: 0, 1: 1, 2: 0, 3: 2}
Test Case 2 - Path: {0: [0], 1: [0], 2: [0, 1, 2], 3: [0, 2, 3]}
{0: 0, 1: 1, 2: -2, 3: -1}
{0: 0, 1: 1, 2: -2, 3: -1}
Test Case 3 - Distance: {0: 0, 1: 1, 2: -2, 3: -1}
Test Case 3 - Path: {0: [0], 1: [0], 2: [0, 1, 2], 3: [0, 2, 3]}
{0: 0, 1: 1, 2: 3, 3: inf, 4: inf}
{0: 0, 1: 1, 2: 3, 3: inf, 4: inf}
Test Case 4 - Distance: {0: 0, 1: 1, 2: 3, 3: inf, 4: inf}
Test Case 4 - Path: {0: [0], 1: [0], 2: [0, 1, 2], 3: [0], 4: [0]}


In [110]:
graph = WeightedGraph(5)
graph.add_edge(0, 1, -1)
graph.add_edge(0, 2, 4)
graph.add_edge(1, 2, 3)
graph.add_edge(1, 3, 2)
graph.add_edge(1, 4, 2)
graph.add_edge(3, 2, 5)
graph.add_edge(3, 1, 1)
graph.add_edge(4, 3, -3)

source = 0
k = 3  # Adjusting k according to the constraint

distance, path = BellmanFord(graph, source, k)
print("Test Case 2 - Distance:", distance)
print("Test Case 2 - Path:", path)

graph = WeightedGraph(5)
graph.add_edge(0, 1, 1)
graph.add_edge(1, 2, 1)
graph.add_edge(2, 0, -3)
graph.add_edge(2, 3, 2)
graph.add_edge(3, 4, 2)

source = 0
k = 3  # This is the corrected k value

distance, path = BellmanFord(graph, source, k)
print("Test Case 3 - Distance:", distance)
print("Test Case 3 - Path:", path)

{0: 0, 1: -1, 2: 2, 3: -2, 4: 1}
{0: 0, 1: -1, 2: 2, 3: -2, 4: 1}
Test Case 2 - Distance: {0: 0, 1: -1, 2: 2, 3: -2, 4: 1}
Test Case 2 - Path: {0: [0], 1: [0], 2: [0, 1, 2], 3: [0, 1, 3, 4, 3], 4: [0, 1, 4]}
{0: -3, 1: -1, 2: 0, 3: 2, 4: 4}
{0: -4, 1: -2, 2: -1, 3: 1, 4: 3}


ValueError: Negative cycle